In [1]:
import numpy as np
from skimage import io

import os
import napari
import pickle

from sklearn.preprocessing import normalize, PolynomialFeatures
from sklearn import linear_model

import morphotrack.vessel

import importlib
importlib.reload(morphotrack.vessel)


/home/tmurakami/app/miniconda3/envs/segment/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


<module 'morphotrack.vessel' from '/home/tmurakami/src/morphotrack/morphotrack/vessel.py'>

In [2]:
io_directory = '/mnt/ampa_data01/tmurakami/220715_prefrontal_q2_R01/morphotrack'

In [3]:
# get binary data for blood vesssel
vessel = io.imread(os.path.join(io_directory,'vessel_labkit.tif'))==1

In [4]:
# make guide vector from the binary image.
# load a mask of a white matter 
white_matter = io.imread(os.path.join(io_directory,'white_matter.tif'))!=0
# load a mask of a layer 1
layer1 = io.imread(os.path.join(io_directory,'l1.tif'))!=0

# make guide vector 
guide_vector = morphotrack.vessel.guide_vector_generator_from_binaries(layer1, white_matter)

# or manually make a guide vector
# guide_coordinate1 = np.array([155.,122.,197.])
# guide_coordinate2 = np.array([110.,315.,213.])
# guide_vector = morphotrack.vessel.guide_vector_generator(guide_coordinate1, guide_coordinate2)

In [5]:
vectors, img = morphotrack.vessel.get_vectors_from_vessel(vessel, guide_vector, threshold=10, k=27, return_image=True)

start skeletonization
get vectors from skeletonized image
expand vectors to original image


In [ ]:
# visualize if needed
viewer = napari.Viewer()
for i, c in enumerate(['red','green','blue']):
    viewer.add_image(img[...,i], contrast_limits=[0,1], rgb=False, name='vessel', colormap=c, blending='additive')

In [6]:
# Extract the vectors and positions using vessel.
vessel_position = np.array(np.where(vessel)).T

# The first smoothing
smoothed_vectors = morphotrack.vessel.smooth_vectors(vessel_position,vectors)

(get_neighbor_vectors pid=4057718) 
(get_neighbor_vectors pid=4057728) 
(get_median_vector pid=4057671) 
(get_neighbor_vectors pid=4057684) 
(get_neighbor_vectors pid=4057700) 
(get_neighbor_vectors pid=4057658) 
(get_median_vector pid=4059567) 
(get_median_vector pid=4057700) 
(get_neighbor_vectors pid=4057708) 
(get_neighbor_vectors pid=4059662) 
(get_median_vector pid=4059496) 
(get_neighbor_vectors pid=4057685) 
(get_median_vector pid=4057720) 
(get_neighbor_vectors pid=4057690) 
(get_median_vector pid=4058109) 
(get_median_vector pid=4058647) 
(get_median_vector pid=4059783) 
(get_median_vector pid=4057680) 
(get_median_vector pid=4057694) 
(get_median_vector pid=4059911) 
(get_neighbor_vectors pid=4059836) 
(get_median_vector pid=4060025) 
(get_neighbor_vectors pid=4057660) 
(get_median_vector pid=4058109) 
(get_neighbor_vectors pid=4057646) 
(get_neighbor_vectors pid=4057680) 
(get_median_vector pid=4060091) 
(get_neighbor_vectors pid=4058647) 
(get_neighbor_vectors pid=4059758)

In [ ]:
# visualize if needed
img = np.zeros(vessel.shape+(vessel.ndim,))
img[tuple(vessel_position.T)] = smoothed_vectors

viewer = napari.Viewer()
for i, c in enumerate(['red','green','blue']):
    viewer.add_image(img[...,i], contrast_limits=[0,1], rgb=False, name='vessel', colormap=c, blending='additive')

In [7]:
# remove sub-branches for the better fitting
# dots = (smoothed_vectors * vectors).sum(axis=1)
# thresh = threshold_otsu(dots)
keep = morphotrack.vessel.remove_dissimilar_vectors(smoothed_vectors, vectors, threshold='otsu') # dots>=thresh

In [ ]:
# visualize if needed

viewer = napari.Viewer()
img = np.zeros(vessel.shape)
img[tuple(vessel_position[keep].T)] = 1
viewer.add_image(img, contrast_limits=[0,1], rgb=False, name='keep', colormap='green', blending='additive')
img = np.zeros(vessel.shape)
img[tuple(vessel_position[~keep].T)] = 1
viewer.add_image(img, contrast_limits=[0,1], rgb=False, name='keep', colormap='magenta', blending='additive')

In [8]:
%%time
# repeat the process to sophisticate the vector smoothing
vessel_position = np.array(np.where(vessel)).T[keep]

# The second smoothing
smoothed_vectors = morphotrack.vessel.smooth_vectors(vessel_position,vectors[keep])

(get_neighbor_vectors pid=4065854) 
(get_neighbor_vectors pid=4066555) 
(get_neighbor_vectors pid=4065923) 
(get_median_vector pid=4066367) 
(get_neighbor_vectors pid=4062558) 
(get_median_vector pid=4065335) 
(get_median_vector pid=4064133) 
(get_neighbor_vectors pid=4064602) 
(get_neighbor_vectors pid=4066270) 
(get_neighbor_vectors pid=4066270) 
(get_neighbor_vectors pid=4065923) 
(get_median_vector pid=4064395) 
(get_median_vector pid=4066036) 
(get_median_vector pid=4064395) 
(get_median_vector pid=4066170) 
(get_median_vector pid=4064798) 
(get_median_vector pid=4066367) 
(get_median_vector pid=4065714) 
(get_median_vector pid=4065714) 
(get_neighbor_vectors pid=4064373) 
(get_neighbor_vectors pid=4066170) 
(get_median_vector pid=4065736) 
(get_median_vector pid=4064133) 
(get_median_vector pid=4066170) 
(get_neighbor_vectors pid=4065802) 
(get_neighbor_vectors pid=4066270) 
(get_median_vector pid=4067172) 
(get_neighbor_vectors pid=4064133) 
(get_median_vector pid=4066840) 
(get

In [9]:
# Fit to the nth polynomial
degree = 5 # Overfitting may happen at the edge if degree is too high
vec = smoothed_vectors
poly = PolynomialFeatures(degree=degree) 
idx_ = poly.fit_transform(vessel_position)

reg = linear_model.LinearRegression(fit_intercept=True) # True
reg.fit(idx_,vec)# Fit the model
reg.degree = degree# save information for polynomial degree for later use

In [10]:
# visualize if needed
img = np.zeros(vessel.shape+(vessel.ndim,))
img[tuple(vessel_position.T)] = smoothed_vectors

viewer = napari.Viewer()
for i, c in enumerate(['red','green','blue']):
    viewer.add_image(img[...,i], contrast_limits=[0,1], rgb=False, name='vessel', colormap=c, blending='additive')


pred = normalize(reg.predict(idx_), axis=1)
img = np.zeros(vessel.shape+(vessel.ndim,))
img[tuple(vessel_position.T)] = pred

for i, c in enumerate(['red','green','blue']):
    viewer.add_image(img[...,i], contrast_limits=[0,1], rgb=False, name='prediction', colormap=c, blending='additive')

In [ ]:
pickle.dump(reg, open(os.path.join(io_directory,'model_.pkl'), 'wb'))